# Exploring Hackers News Posts

In this project, we'll compare two different types of posts from [Hacker News](https://news.ycombinator.com/), a popular site where technology related stories (or 'posts') are voted and commented uon. The two types of posts we'll explore begin with either `Ask HN` or `Show HN`.

![img](hacker_news_logo.jpg)

User submit `Ask HN` posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, user submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

We's specifically compare these two types of posts to determine the following:
- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

# Introduction

First, we'll read in the data and remove the headers.

In [1]:
# read in the data and remove headers

import csv
f = open('hacker_news.csv')
hn = list(csv.reader(f))
hn[:5] # print first 5 rows

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing headers from a list of lists

In [2]:
# remove header row
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see that the data set contains the following columns:
- `title`: title of the post (self explanatory)
- `url`: the url of the item being linked to
- `num_points`: the number of upvotes the post received
- `num_comments`: the number of comments the post received
- `author`: the name of the account that made the post
- `created_at`: the date and time the post was made (the time zone is Eastern Time in the US)

Let's start by exploring the number of comments for each type of post.

## Extracting 'Ask HN' and 'Show HN' posts

First, we'll identify posts that begin with either `Ask HN` or `Show HN` and seperate the data for those two types of posts into different lists.

In [3]:
# identify posts that begin with either 'Ask HN' or 'Show HN' in the 'title' columnand seperate them into differetn lists

ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(post)
    elif title.lower().startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
# check how many posts are for each list
print('Ask HN has', len(ask_posts), 'posts')
print('Show HN has',len(show_posts),'posts')
print('Other posts has',len(other_posts),'posts')

Ask HN has 1744 posts
Show HN has 1162 posts
Other posts has 17194 posts


## Calculating the averaage number of comments for 'Ask HN' and 'Show HN' posts

In [4]:
# calculate the average number of comments 'Ask HN' posts receive

total_ask_comments = 0

for post in ask_posts:
    number_of_comments = int(post[4])
    total_ask_comments += number_of_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)
    

14.038417431192661


In [5]:
# calculate the average number of comments 'Show HN' posts receive

total_show_comments = 0

for post in show_posts:
    number_of_comments = int(post[4])
    total_show_comments += number_of_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts in our sample receive approximately `14` comments, whereas show posts receive approximately 10. Since ask posts are more likely to receieve comments, we'll focus our remaining analysis just on these posts.

## Finding the amount of 'Ask Posts' and comments by Hour created

Next, we'll determine if we can maximise the amount of comments an ask post receives by creating it at a certain time. 
- First, we'll find the amount of ask posts created during each hor of day, along with the number of comment those posts received. 
- Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [6]:
# calculate the amount ask posts created during each hour of the day and the number of comments received
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append([post[6], int(post[4])])
    
comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    # each_row[0] = time
# print(result_list)
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1
        
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the average number of comments for 'Ask HN' posts by hour

In [7]:
# calculating the average amount of comments 'Ask HN' posts created at each hour of the day receive

avg_com_by_hour = []

for hr in comments_by_hour:
    avg_com_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_com_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

### Sorting and printing values of avg_com_by_hour (list of lists)

In [8]:
sorted_avg_by_hr = []

# swap columns
for row in avg_com_by_hour:
    sorted_avg_by_hr.append([row[1],row[0]])
    
# sort column
sorted_result = sorted(sorted_avg_by_hr,reverse=True)

sorted_result

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [9]:
# select the 5 hours with the highest average amount of comments

print("Top 5 hours for 'Ask HN' comments")
for avg_comments, hr in sorted_result[:5]:
    print(("{} has average comments of {:.2f} per post").format(dt.datetime.strptime(hr, "%H").strftime("%H:%M"), avg_comments))

Top 5 hours for 'Ask HN' comments
15:00 has average comments of 38.59 per post
02:00 has average comments of 23.81 per post
20:00 has average comments of 21.52 per post
16:00 has average comments of 16.80 per post
21:00 has average comments of 16.01 per post


In [10]:
# calculate the difference in average comments between the highest and the second highest

top1_vs_top2 = round((sorted_result[0][0] - sorted_result[1][0]) / sorted_result[1][0] *100)
print(str(top1_vs_top2) + "%")


62%


The hour that receives the most comments per post on average is `15:00` with an average of 38.59 comments per post. There is about `62%` increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. so we could also say 15:00 as 3:00 pm est.

## Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comment on average. Based on the analysis above, to maximise the amount of comments a post can receive, we'd recommend the post be categorised as ask post and posted between `15:00 and 16:00 (3:00 pm est - 4:00 pm est)`

It should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that ***'of the posts that received comments'***, ask posts received more comments on average and ask posts created around 15:00 (3:00 pm est) received the most comments on average.